## References

* https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai/blob/main/05-continuous-training.ipynb

In [ ]:
!gcloud init

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
GOOGLE_CLOUD_PROJECT = 'fast-ai-exploration'    
GOOGLE_CLOUD_REGION = 'us-central1'      
GCS_BUCKET_NAME = 'vertex-tfx-mlops'     

PIPELINE_NAME = 'penguin-vertex-training'
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)
PIPELINE_LOCATION = f'{PIPELINE_ROOT}/{PIPELINE_NAME}.json'
PUBSUB_TOPIC = f'trigger-{PIPELINE_NAME}'

DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

In [ ]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

In [ ]:
ENV_VARS=f"""\
GCP_PROJECT={GOOGLE_CLOUD_PROJECT},\
GCP_REGION={GOOGLE_CLOUD_REGION},\
GCS_PIPELINE_FILE_LOCATION={PIPELINE_LOCATION}
"""

!echo {ENV_VARS}

In [ ]:
!git clone -b dev https://github.com/sayakpaul/CI-CD-for-Model-Training
%cd CI-CD-for-Model-Training

In [ ]:
BUCKET = f'gs://{GCS_BUCKET_NAME}'
!gcloud functions deploy {CLOUD_FUNCTION_NAME} \
    --region={REGION} \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python37 \
    --source=cloud_function\
    --entry-point=trigger_pipeline\
    --stage-bucket={BUCKET}\
    --update-env-vars={ENV_VARS}

# `trigger_pipeline` is the name of the function inside
# `cloud_function/main.py`

In [ ]:
from google.cloud import pubsub
import tensorflow as tf

publish_client = pubsub.PublisherClient()
topic = f'projects/{PROJECT}/topics/{PUBSUB_TOPIC}'
data = {
    'num_epochs': 5,
    'batch_size': 64,
    'optimizer': tf.keras.optimizers.Adam(learning_rare=1e-2)
}
message = json.dumps(data)

_ = publish_client.publish(topic, message.encode())